### Visualizing ICESat-2 data

Instructors: [Tyler Sutterley](mailto:tsutterl@uw.edu), Nathan Kurtz and Ben Smith

This notebook uses standard python tools to demonstrate some basic visualization of ICESat-2 products

In [ ]:
from __future__ import print_function, division

import os
os.environ['USE_PYGEOS'] = '0'
import re
import logging
import numpy as np
import matplotlib.pyplot as plt
import icesat2_toolkit as is2tk
import IS2view
# autoreload
%load_ext autoreload
%autoreload 2
# set logging level
logging.basicConfig(level=logging.ERROR)

#### Create s3 filesystem with `s3fs`

In [ ]:
client = is2tk.utilities.attempt_login('urs.earthdata.nasa.gov',
    authorization_header=True)
session = is2tk.utilities.s3_filesystem()

#### ATLAS Background
The primary and secondary instrumentation onboard the ICESat-2 observatory are the Advanced Topographic Laser Altimeter System (ATLAS, a photon-counting laser altimeter), the global positioning system (GPS) and the star cameras. 
Data from these instruments are combined to create three primary measurements: the time of flight of a photon transmitted and received from ATLAS, the position of the satellite in space, and the pointing vector of the satellite during the transmission of photons. 
These three measurements are used to create ATL03, the geolocated photon product of ICESat-2.  

#### ATL03 - Global Geolocated Photon Data
- Precise latitude, longitude and elevation for every received photon, arranged by beam in the along-track direction  
- Photons classified by signal vs. background, as well as by surface type (land ice, sea ice, land, ocean), including all geophysical corrections  

More information about ATL03 can be found in the Algorithm Theoretical Basis Documents (ATBDs) provided by the ICESat-2 project:  
- [ATL03: Global Geolocated Photon Data](https://nsidc.org/sites/default/files/documents/technical-reference/icesat2_atl03_atbd_v006.pdf)  
- [ATL03g: Received photon geolocation](https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/ICESat2_ATL03g_ATBD_r002.pdf)  
- [ATL03a: Atmospheric Delay Corrections](https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/I2_ATL03A_ATBD.pdf)  

#### Read ATL03 HDF5 file for root variables and attributes
The structure of the file has six groups for each beam, data describing the responses of the ATLAS instrument, ancillary data for correcting and transforming the ATL03 data, and a group of metadata.  

In [ ]:
# query CMR for path to Release-06 granule
ATL03_id, ATL03_s3_url = is2tk.utilities.cmr(product='ATL03',
    release=6, cycles=5, tracks=752, granules=12,
    provider=is2tk.utilities._s3_providers['nsidc'],
    endpoint='s3', request_type='application/x-hdf5')
# access ATL03 file from NSIDC
ATL03_granule = session.open(ATL03_s3_url[0], mode='rb')

# load ICESat-2 ATL03 data
IS2_atl03_mds,IS2_atl03_attrs,IS2_atl03_beams = \
    is2tk.io.ATL03.read_main(ATL03_granule, ATTRIBUTES=True)

# extract parameters from ICESat-2 ATLAS HDF5 file name
rx = re.compile(r'(processed_)?(ATL\d{2})_(\d{4})(\d{2})(\d{2})(\d{2})'
    r'(\d{2})(\d{2})_(\d{4})(\d{2})(\d{2})_(\d{3})_(\d{2})(.*?).h5$')
SUB,PRD,YY,MM,DD,HH,MN,SS,TRK,CYCL,GRAN,RL,VERS,AUX = rx.findall(ATL03_id[0]).pop()

# number of GPS seconds between the GPS epoch
# and ATLAS Standard Data Product (SDP) epoch
atlas_sdp_gps_epoch, = IS2_atl03_mds['ancillary_data']['atlas_sdp_gps_epoch']

#### Create a scatter plot of photon elevations vs time

ATL03 contains most of the data needed to create the higher level data products. 

The ATL03 photon events will have a confidence level flag associated with it for a given surface type:  
- -2: possible Transmit Echo Path photons  
- -1: events not associated with a specific surface type  
- 0: noise  
- 1: buffer but algorithm classifies as background  
- 2: low  
- 3: medium  
- 4: high  

In the confidence level matrix, the column of each surface types is:   
- 0: Land  
- 1: Ocean  
- 2: Sea Ice  
- 3: Land Ice  
- 4: Inland Water  

The level of confidence of a given photon event (PE) may vary based on the surface type

This particular set of data approaches the grounding line near Thwaites Glacier in West Antarctica, has some photon events that are impacted by the presence of clouds (highly scattered lower confidence photon events), and has some possible Transmitter Echo Photons (TEP) (the red curved line of anomalous photon events in gt2r and gt3r).  These are [things to be aware of when analyzing photon event data from ATL03](https://nsidc.org/sites/nsidc.org/files/technical-references/ATL03_Known_Issues_May2019.pdf).  

The ATLAS instrument decides whether or not to telemeter packets of received photons back as data.  ATLAS uses a digital elevation model (DEM) and a few rules to decide whether to transmit large blocks of data to NASA.  The telemetry bands are evident by the spread of low confidence photon events around the surface for each beam.  

Photon events in ATL03 can come to the ATLAS receiver in a few different ways:  
- Many photons come from the sun either by reflecting off clouds or the land surface.  These photon events are spread in a random distribution along the telemetry band.  In ATL03, a large majority of these "background" photon events are classified, but some may be incorrectly classified as signal.  
- Some photons are from the ATLAS instrument that have reflected off clouds. These photons can be clustered together or widely dispersed depending on the properties of the cloud and a few other variables.  
- Some photons will be returns from the [Transmit Echo Path (TEP)](https://nsidc.org/sites/nsidc.org/files/technical-references/ATL03_Known_Issues_May2019.pdf)  
- Some photons are from the ATLAS instrument that have reflected off the surface (our signal photons).  

There will be photons transmitted by the ATLAS instrument will never be recorded back.  The vast majority of these photons never reached the ATLAS instrument again  (only about 10 out of the 10^<sup>14</sup> photons transmitted are received), but some are not detected due to the "dead time" of the instrument.  This can create a bias towards the first photons that were received by the instrument.  This first photon bias (FPB) is estimated in the higher level data products.  

The transmitted pulse is also not symmetric in time, which can introduce a bias when calculating average surfaces.  The magnitude of this bias depends on the shape of the transmitted waveform, the width of the window used to calculate the average surface, and the slope and roughness of the surface that broadens the return pulse.  This transmit-pulse shape bias is also estimated in the higher level data products.  

To show the impacts of clouds in more detail, we can investigate the confidence of each photon event for a cloud impacted beam.  For this file, the cloud is situated over the sea surface and does not have an associated confidence for land ice.

In [ ]:
%matplotlib inline
# create scatter plot of photon data versus along-track distance
f1,ax1 = plt.subplots(num=1,nrows=2,sharex=True,sharey=True,figsize=(10,10))

# data for beam gtx
gtx = 'gt2r'
val,attrs = is2tk.io.ATL03.read_beam(ATL03_granule, gtx, ATTRIBUTES=True)

# ATL03 Segment ID
Segment_ID = val['geolocation']['segment_id']
# number of photon events
n_pe, = val['heights']['delta_time'].shape
# first photon in the segment (convert to 0-based indexing)
Segment_Index_begin = val['geolocation']['ph_index_beg'] - 1
# number of photon events in the segment
Segment_PE_count = val['geolocation']['segment_ph_cnt']
# along-track distance for each ATL03 segment
Segment_Distance = val['geolocation']['segment_dist_x']
# along-track length for each ATL03 segment
Segment_Length = val['geolocation']['segment_length']
# along-track and across-track distance for photon events
x_atc = val['heights']['dist_ph_along']
y_atc = val['heights']['dist_ph_across']
# photon event heights
h_ph = val['heights']['h_ph']

# for each 20m segment
for j,_ in enumerate(Segment_ID):
    # index for 20m segment j
    idx = Segment_Index_begin[j]
    # skip segments with no photon events
    if (idx < 0):
        continue
    # number of photons in 20m segment
    cnt = Segment_PE_count[j]
    # add segment distance to along-track coordinates
    x_atc[idx:idx+cnt] += Segment_Distance[j]

# check confidence level associated with each photon event
# -1: Events not associated with a specific surface type
#  0: noise
#  1: buffer but algorithm classifies as background
#  2: low
#  3: medium
#  4: high
# Signal classification confidence for land ice
# 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
ice_sig_conf = val['heights']['signal_conf_ph'][:,3]
# find possible TEP events
isTEP, = np.nonzero(ice_sig_conf == -2)
# find different surface classification photon events
stype, = np.nonzero(ice_sig_conf == -1)
# background and buffer photons
bg, = np.nonzero((ice_sig_conf == 0) | (ice_sig_conf == 1))
# find photon events of progressively higher confidence
lc, = np.nonzero(ice_sig_conf == 2)
mc, = np.nonzero(ice_sig_conf == 3)
hc, = np.nonzero(ice_sig_conf == 4)
# Photon event geolocation and elevation (WGS84)
ax1[0].plot(x_atc[isTEP], h_ph[isTEP], marker='.',
    markersize=0.1, lw=0, color='red', label='TEP')
ax1[0].plot(x_atc[stype], h_ph[stype], marker='.',
    markersize=0.1, lw=0, color='0.2', label='Surface Classification')
ax1[0].plot(x_atc[bg], h_ph[bg], marker='.',
    markersize=0.1, lw=0, color='0.5', label='Background')
ax1[0].plot(x_atc[lc], h_ph[lc], marker='.',
    markersize=0.1, lw=0, color='darkorange', label='Low Confidence')
ax1[0].plot(x_atc[mc], h_ph[mc], marker='.',
    markersize=0.1, lw=0, color='mediumseagreen', label='Medium Confidence')
ax1[0].plot(x_atc[hc], h_ph[hc], marker='.',
    markersize=0.1, lw=0, color='darkorchid', label='High Confidence')

# YAPC photon event classifier
weight_ph = val['heights']['weight_ph']
isort = np.argsort(weight_ph)
sc = ax1[1].scatter(x_atc[isort], h_ph[isort], c=weight_ph[isort], s=0.1)
# add colorbar for scatter plot
cax = f1.add_axes([0.075, 0.080, 0.305, 0.02])
# add extension triangles to upper and lower bounds
# pad = distance from main plot axis
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = f1.colorbar(sc, cax=cax, extend='both', extendfrac=0.0375,
    drawedges=False, orientation='horizontal')
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_xlabel('Photon Classifier SNR')
cbar.ax.xaxis.set_label_position('top')
# ticks lines all the way across
cbar.ax.tick_params(which='both',width=1,length=15,direction="in")

# set title and labels
ax1[1].set_xlabel('Along-Track Distance [m]')
ax1[0].set_ylabel('Elevation above WGS84 Ellipsoid [m]')
ax1[1].set_ylabel('Elevation above WGS84 Ellipsoid [m]')
ax1[0].set_title(f'{PRD} RGT:{TRK} Cycle: {CYCL} Region: {GRAN} GT: {gtx}')
# create legend
lgd = ax1[0].legend(loc=3,frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)
    
# adjust the figure axes
f1.subplots_adjust(left=0.07, right=0.98, bottom=0.05, top=0.95, hspace=0.05)
# show the plot
plt.show()

#### ATL06 - Land Ice Height Data
- Latitude, longitude and elevation for overlapping 40m along-track segments  
- cm-level corrections for known instrument biases
- Ancillary parameters that can be used to interpret and assess the quality of the height estimates

More information about ATL06 can be found in the [ATBD](https://nsidc.org/sites/default/files/documents/technical-reference/icesat2_atl06_atbd_v006.pdf) provided by NSIDC


In [ ]:
# query CMR for path to Release-06 granule
ATL06_id, ATL06_s3_url = is2tk.utilities.cmr(product='ATL06',
    release=6, cycles=5, tracks=752, granules=12,
    provider=is2tk.utilities._s3_providers['nsidc'],
    endpoint='s3', request_type='application/x-hdf5')
# access ATL06 file from NSIDC
ATL06_granule = session.open(ATL06_s3_url[0], mode='rb')

# load ICESat-2 ATL06 data
IS2_atl06_mds,IS2_atl06_attrs,IS2_atl06_beams = \
    is2tk.io.ATL06.read_granule(ATL06_granule, ATTRIBUTES=True)

# extract parameters from ICESat-2 ATLAS HDF5 file name
rx = re.compile(r'(processed_)?(ATL\d{2})_(\d{4})(\d{2})(\d{2})(\d{2})'
    r'(\d{2})(\d{2})_(\d{4})(\d{2})(\d{2})_(\d{3})_(\d{2})(.*?).h5$')
SUB,PRD,YY,MM,DD,HH,MN,SS,TRK,CYCL,GRAN,RL,VERS,AUX = rx.findall(ATL06_id[0]).pop()

In [ ]:
%matplotlib widget
# create scatter plot of segment data versus along-track distance
f2,ax2 = plt.subplots(num=2, figsize=(10,10))

# data for beam gtx
gtx = 'gt2r'
val = IS2_atl06_mds[gtx]['land_ice_segments']
attrs = IS2_atl06_attrs[gtx]['land_ice_segments']

# ATL06 Segment ID
Segment_ID = val['segment_id']
# number of ATL06 segments
n_seg, = val['delta_time'].shape
# average transmit time of the segment
delta_time = val['delta_time']
# along-track and across-track distance 
x_atc = val['ground_track']['x_atc']
y_atc = val['ground_track']['y_atc']
dh_dx = val['fit_statistics']['dh_fit_dx']
dx = 20.0
# land ice heights
h_li = np.ma.array(val['h_li'], fill_value=attrs['h_li']['_FillValue'])
h_li.mask = (h_li.data == h_li.fill_value)
# segment quality summary flag
atl06_quality_summary = val['atl06_quality_summary']
# find low and high quality data
lq, = np.nonzero(atl06_quality_summary != 0)
hq, = np.nonzero(atl06_quality_summary == 0)

# segment geolocation and elevation (WGS84)
ax2.plot(x_atc[lq], h_li[lq], marker='.',
    markersize=5, lw=0, color='mediumseagreen', label='Low-Quality Data')
ax2.plot(x_atc[hq], h_li[hq], marker='.',
    ms=5, lw=0, color='darkorchid', label='High-Quality Data')
distances = np.c_[x_atc[hq] - dx, x_atc[hq] + dx]
heights = np.c_[h_li[hq] - dx*dh_dx[hq], h_li[hq] + dx*dh_dx[hq]]
ax2.plot(distances.T, heights.T, color='0.5', alpha=0.5, lw=1)

# set title and labels
ax2.set_xlabel('Along-Track Distance [m]')
ax2.set_ylabel('Elevation above WGS84 Ellipsoid [m]')
ax2.set_title(f'{PRD} RGT:{TRK} Cycle: {CYCL} Region: {GRAN} GT: {gtx}')
# create legend
lgd = ax2.legend(loc=3,frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)
    
# adjust the figure axes
f2.subplots_adjust(left=0.07, right=0.98, bottom=0.05, top=0.95)
# show the plot
plt.show()

#### ATL11 - Slope-Corrected Land Ice Height Time Series Data
- Latitude, longitude and elevation for 120m along-track segments  

Data is combined from the two beam pairs into a single pair track solution, corrected for across-track slope. ATL11 height data is easily comparable through time to calculate the elevation change. More information about ATL11 can be found in the [ATBD](https://nsidc.org/sites/default/files/icesat2_atl11_atbd_r005_0.pdf) provided by NSIDC

In [ ]:
# query CMR for path to ATL11 Release-05 granule
ATL11_id, ATL11_s3_url = is2tk.utilities.cmr(product='ATL11',
    release=5, tracks=752, granules=12,
    provider=is2tk.utilities._s3_providers['nsidc'],
    endpoint='s3', request_type='application/x-hdf5')
# access ATL11 file from NSIDC
ATL11_granule = session.open(ATL11_s3_url[0], mode='rb')
    
# load ICESat-2 ATL11 data
IS2_atl11_mds,IS2_atl11_attrs,IS2_atl11_pairs = is2tk.io.ATL11.read_granule(
    ATL11_granule, ATTRIBUTES=True)

# extract parameters from ICESat-2 ATLAS HDF5 file name
rx = re.compile(r'(processed_)?(ATL\d{2})_(\d{4})(\d{2})_(\d{2})(\d{2})_'
    r'(\d{3})_(\d{2})(.*?).h5$')
SUB,PRD,TRK,GRAN,SCYC,ECYC,RL,VERS,AUX = rx.findall(ATL11_id[0]).pop()

In [ ]:
%matplotlib widget
# create scatter plot of segment data versus along-track distance
f3,ax3 = plt.subplots(num=3, figsize=(10,10))

# data for beam pair ptx
ptx = 'pt2'
val = IS2_atl11_mds[ptx]
attrs = IS2_atl11_attrs[ptx]

# ATL11 reference points
Segment_ID = val['ref_pt']
# number of ATL11 segments
n_seg,n_cycles = val['delta_time'].shape
# average transmit time of the segment
delta_time = val['delta_time']
# longitude and latitude
longitude = val['longitude']
latitude = val['latitude']
# slope corrected land ice heights
h_corr = np.ma.array(val['h_corr'], fill_value=attrs['h_corr']['_FillValue'])
h_corr.mask = (h_corr.data == h_corr.fill_value)
# only include high-quality segments
h_corr.mask |= (val['quality_summary'] != 0)

# segment geolocation and elevation for each cycle
colors = plt.cm.viridis(np.linspace(0,1,n_cycles))
for i,c in enumerate(val['cycle_number']):
    # along-track and across-track distance
    x_atc = val['cycle_stats']['x_atc'][:,i]
    ax3.plot(x_atc, h_corr[:,i], marker='.',
        ms=5, lw=0, color=colors[i], label=f'Cycle {c:d}')

# set title and labels
ax3.set_xlabel('Along-Track Distance [m]')
ax3.set_ylabel('Elevation above WGS84 Ellipsoid [m]')
ax3.set_title(f'{PRD} RGT:{TRK} Region: {GRAN} PT: {ptx}')
# create legend
lgd = ax3.legend(loc=3, frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)
    
# adjust the figure axes
f3.subplots_adjust(left=0.07, right=0.98, bottom=0.05, top=0.95)
# show the plot
plt.show()

#### ATL15 -Gridded Land Ice Height Change Data
- Time-variable land ice height change and change rate at 1km, 10km, 20km and 40km spatial resolutions
- Relative elevation provided at quarter-annual time resolution
- Height change rate at quarter-annual, annual and multi-annual time resolutions

All ATL11 along-track data is combined in a constrained least-squares solution to provided gridded fields that are smoothly interpolated between tracks. The solution provides the reference elevation at a time point (ATL14) and the relative elevation change at the quarter-annual time points (ATL15). More information about ATL14 and ATL15 can be found in the [ATBD](https://nsidc.org/sites/default/files/documents/technical-reference/icesat2_atl14_atl15_atbd_r002.pdf) provided by NSIDC

In [ ]:
# query CMR for path to ATL15 Release-02 granule
ATL15_id, ATL15_s3_url = IS2view.utilities.cmr(product='ATL15',
    release=2, regions='AA', resolutions='01km',
    provider=IS2view.utilities._s3_providers['nsidc'],
    endpoint='s3', request_type='application/x-netcdf')
# access ATL15 file from NSIDC
# ATL15_granule = session.open(ATL15_s3_url[0], mode='rb')
ATL15_granule = f'IS2view/notebooks/{ATL15_id[0]}'
# read ATL15 granule
ds = IS2view.io.from_file(ATL15_granule, group='delta_h', format='nc')
ds

In [ ]:
%matplotlib inline
# create geoJSON-like data from ATL11 coordinates
geometry = dict(type='LineString', coordinates=list(zip(longitude, latitude)))
feature = dict(type='Feature', id=0, geometry=geometry)
# plot ATL15 relative heights (in reference to 2020.0) along the ATL11 line
ds.timeseries.plot(feature, variable='delta_h', cmap=plt.cm.viridis, legend=True)